In [1]:
from functions import *
from random import sample
from htmldate import find_date
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import json
from langid.langid import LanguageIdentifier, model
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
from purl import URL

In [2]:
base_path = "/media/ruben/Data Drive/react-data/protest/selection2"


# Get Publication Dates

dates_ref = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    with open(os.path.join(photo_folder,"dates.txt"),'r') as f:
        x = f.readlines()
    dates_ref.update({d.split('|')[0]:d.split('|')[-1].replace('\n','') for d in x if d.split('|')[-1].replace('\n','') != "na" and "ERROR" not in d.split('|')[-1].replace('\n','')})

# Get URL-ID from entity files
ids_ = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    
    for f in folders:
        fn = os.path.join(photo_folder, f, "txt","parsed_text.json")

        with open(fn) as fp:
            file = json.load(fp)
        
        for id_ in file.keys():
            
            identif = id_.split("html_")[0].split('html/')[-1][:-1]
            url = id_.split("html_")[1]
            ids_.update({identif:url})

In [3]:
tlds = []

for id_,date in dates_ref.items():
    year = date[0:4]
    tld = URL.from_string(id_).domain()
    tlds.append([tld,year,1])

In [25]:
# # Plot Diachronic, doesn't work
# tlds = pd.DataFrame(tlds,columns=['tld','year','count']).groupby(['tld','year']).sum().reset_index()
# tlds = tlds[tlds['count'] > 1]
# tlds = tlds.pivot(index='year',columns='tld',values='count').reset_index().fillna(0)
# tlds.plot(x='year',y=list(tlds.columns)[1:],figsize=(20,10))

In [ ]:
# prefixes = ['www']
# extensions = [t[0].split('.')[-1] for t in tlds]

In [11]:
pd.DataFrame(Counter([t[0] for t in tlds]).most_common(50),columns=['tld','n']).to_csv('/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_2/top-level-domains.csv',index=False)